# PyTorch入门

随着神经网络，特别是深度神经网络的出现，如何使用计算机高效的完成参数量巨大的神经网络的训练就成了一个难题。为此，出现了诸如TensorFlow、PyTorch等深度学习框架。在工业界中，TensorFlow出现的早而且应用较多，而PyTorch虽然出现的晚，不过由于其与Python结合更加紧密，且具有动态图等一系列优良特性，我们在这里以PyTorch为例讲解深度学习框架的使用。

在这一节中，我们首先学习PyTorch的一些基础操作。

# Tensor的使用

在机器学习中，Tensor是向量、矩阵的一个自然推广，即一个任意维的数组，比如：标量、向量（1维数组）、矩阵（2维数组）以及更高维的数组都可以看作是Tensor。

## Tensor的创建

在PyTorch中提供了Tensor数据类型，其使用方法与NumPy类似，但是也有一些差别。比如，为了创建一个Tensor，使用torch中的 **tensor()** 函数就可以了，这与NumPy里面的numpy.array()函数用法是一样的：

In [1]:
import torch
a=torch.tensor([1.,2,3,4])
a

tensor([1., 2., 3., 4.])

也可以通过NumPy的array创建：

In [2]:
import numpy as np
b=np.random.random(5)
b_torch=torch.from_numpy(b)
b_torch

tensor([0.0117, 0.1346, 0.4583, 0.1797, 0.0012], dtype=torch.float64)

或者使用torch.tensor()函数，两者的区别是使用b=torch.from_numpy(a)函数引入时，a和b是共享内存的；而如果使用b=torch.tensor(a)函数时，如果a的类型不是Float32，则会新建，此时可能不共享内存。

反过来，也可以使用Tensor的numpy()函数转化为NumPy的array，此时也是共享内存的：

In [3]:
a=torch.tensor([1.,2,3,4])
b=a.numpy()
print("b=",b)
b[0]=10
print("b=",b)
print("a=",a)

b= [ 1.  2.  3.  4.]
b= [ 10.   2.   3.   4.]
a= tensor([10.,  2.,  3.,  4.])


或者，使用 **Tensor()** 函数（*注意大小写！*）需要创建的Tensor的维数，从而创建一个未初始化的tensor：

In [4]:
c=torch.Tensor(10,3)
c

tensor([[3.5122e-37, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 3.6431e-37, 0.0000e+00],
        [3.6431e-37, 0.0000e+00, 3.6431e-37],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.8395e+25, 6.1963e-04, 1.3119e-08],
        [2.0922e+23, 2.1344e-07, 8.2640e+20],
        [1.7298e-04, 1.3542e-05, 3.6431e-37],
        [0.0000e+00, 3.6431e-37, 0.0000e+00],
        [3.6431e-37, 0.0000e+00, 1.4013e-45],
        [0.0000e+00, 1.8788e+31, 7.9303e+34]])

注意以上新创建的Tensor的值是凌乱的，因为没有对其进行初始化，所以里面的值是完全无意义的。

几乎所有的神经网络工具箱都需要支持GPU计算，而GPU计算最常用的工具是NVIDIA公司的CUDA工具箱。如果你有支持CUDA的显卡，并且配置好了CUDA，可以在创建时通过 **cuda()** 函数将Tensor放在显卡的内存（显存）中，而非内存中：

In [5]:
ac=torch.tensor([1.,2,3,4]).cuda()
ac

tensor([1., 2., 3., 4.], device='cuda:0')

上面的结果除了展示了Tensor的取值之外，还有一个 *device='cuda:0'* ，显示了这个Tensor所出的CUDA设备编号为0。如果有不止一块GPU，可以在括号中加入数字（从0开始计数），指定将其放在哪一块GPU上。

我们可以使用cup()函数和cuda()函数在两者之间转换：

In [6]:
ac_cpu=ac.cpu()
ac_cpu

tensor([1., 2., 3., 4.])

在这里值得一提的是Tensor的数据类型。在PyTorch中，Tensor有如下数据类型可以使用：
* 浮点类型
  * torch.FloatTensor（torch.cuda.FloatTensor）: 32位浮点型
  * torch.DoubleTensor（torch.cuda.DoubleTensor）: 64位浮点型
  * （torch.cuda.HalfTensor）: 16位浮点型
* 整型
  * torch.IntTensor（torch.cuda.IntTensor）: 32位整型
  * torch.LongTensor（torch.cuda.LongTensor）: 64位整型
  * torch.ShortTensor（torch.cuda.ShortTensor）: 16位整型
  * torch.ByteTensor（torch.cuda.ByteTensor）：8位无符号整型
  * torch.CharTensor（torch.cuda.CharTensor）：8位有符号整型
  
其中括号里面表示的是在CUDA中的类型。

不管是在CPU上还是GPU上，位数更小的类型总是需要更少的内存（显存），并且具有更高的计算速度。然而位数更小的类型可能具有更差的精度或者更高的误差（特别是对于浮点类型）、更小的表示范围（特别是对于整数类型），所以在创建Tensor时可以按照需求指定类型。

我们可以使用Tensor.type()函数展示其类型：

In [7]:
print(ac.type())
print(c.type())

torch.cuda.FloatTensor
torch.FloatTensor


根据上面的输出可以知道，默认的类型是32位浮点型，我们可以使用double()（以及float(), int(), long(), short()等）将其换为64位浮点型：

In [8]:
ac_64=ac.double()
print(ac_64.type())
ac_64

torch.cuda.DoubleTensor


tensor([1., 2., 3., 4.], device='cuda:0', dtype=torch.float64)

也可以使用type_as()方法产生相同类型的Tensor：

In [9]:
d=c.double()
e=torch.tensor([1,2,3])
print("e的类型：", e.type())
e_64=e.type_as(d)
print("e_64的类型：", e_64.type())
print(e)
print(e_64)

e的类型： torch.LongTensor
e_64的类型： torch.DoubleTensor
tensor([1, 2, 3])
tensor([1., 2., 3.], dtype=torch.float64)


如果需要改变所有的默认类型，可以使用torch.set_default_tensor_type()函数，比如：

In [10]:
a=torch.tensor([1.,2,3,4])
print(a.type())
torch.set_default_tensor_type("torch.DoubleTensor")
a=torch.tensor([1.,2,3,4])
print(a.type())

torch.FloatTensor
torch.DoubleTensor


一般而言，32位的浮点数已经足够用了，用Double会比较慢，特别是涉及到神经网络中大规模的计算，使用Double的计算开销比较大，所以不妨换回去：

In [11]:
torch.set_default_tensor_type("torch.FloatTensor")

与NumPy类似，PyTorch也有很多创建特殊Tensor的方法，比如：

* 全为0的Tensor：torch.zeors()
* 全为1的Tensor：torch.ones()
* 单位阵：torch.eye()
* U(0,1)的随机数：torch.rand()
* 标准正态分布的随机数：torch.randn()
* 从start到end，步长为step：torch.arange(start,end,step)
* 从start到end，切成m份：torch.linspace(start,end,m)
* 从10^start到10^end，切成m份：torch.linspace(start,end,m)
* 随机排列：torch.randperm(m)

比如：

In [12]:
a=torch.zeros(5,4)
print("维数为5-by-4的全为0的矩阵：\n",a)
b=torch.ones(5,4)
print("维数为5-by-4的全为1的矩阵：\n",b)
c=torch.eye(5,4)
print("维数为5-by-4的单位阵：\n",c)
d=torch.rand(5,4)
print("维数为5-by-4的(0,1)上均匀分布随机数矩阵：\n",d)
e=torch.randn(5,4)
print("维数为5-by-4的标准正态分布矩阵：\n",e)
f=torch.arange(1,10,2)
print("从1到10，步长为2：\n",f)
g=torch.linspace(1,10,5)
print("从1到10，4等分（5个数字）：：\n",g)
h=torch.logspace(0,3,5)
print("从10^1到10^3，4等分（5个数字）：：\n",h)
i=torch.randperm(10)
print("0-9共10个数字的随机排列：\n",i)

维数为5-by-4的全为0的矩阵：
 tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
维数为5-by-4的全为1的矩阵：
 tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
维数为5-by-4的单位阵：
 tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])
维数为5-by-4的(0,1)上均匀分布随机数矩阵：
 tensor([[0.3455, 0.8366, 0.7481, 0.1533],
        [0.5082, 0.4689, 0.2202, 0.0285],
        [0.3153, 0.6423, 0.8443, 0.6514],
        [0.1680, 0.1136, 0.0015, 0.4410],
        [0.7553, 0.4446, 0.2803, 0.1984]])
维数为5-by-4的标准正态分布矩阵：
 tensor([[-0.0331, -1.9236, -1.3173, -0.6466],
        [-0.8283, -0.1833,  0.6490, -1.8499],
        [ 1.0081, -1.4476,  0.4020, -1.1233],
        [ 0.4895,  0.3461,  0.2943,  1.4924],
        [-0.1123,  1.1365, -1.0954, -0.0594]])
从1到10，步长为2：
 tensor([1, 3, 5, 7, 9])
从1到10，4等分（5个数字）：：
 tensor

此外，还可以使用如下命令产生形状相同的Tensor：

* 创建与T形状相同的全为1的Tensor：torch.ones_like(T)
* 创建与T形状相同的全为0的Tensor：torch.zeros_like(T)

In [13]:
j=torch.ones_like(i)
print("与i形状相同的全为1的Tensor：\n",j)
k=torch.zeros_like(g)
print("与g形状相同的全为0的Tensor：\n",k)

与i形状相同的全为1的Tensor：
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
与g形状相同的全为0的Tensor：
 tensor([0., 0., 0., 0., 0.])


与NumPy类似，对于一个Tensor，可以通过shape、size等属性和方法查看Tensor的形状，但是值得注意的是PyTorch的方法、属性与NumPy略有区别：

In [14]:
import numpy as np

np_a=np.random.random((5,6))
print("np_a=\n",np_a)
print("np_a的维度：",np_a.ndim)
print("np_a的形状：",np_a.shape)
print("np_a的长度：",np_a.size)

np_a=
 [[ 0.63123599  0.45494402  0.40702602  0.8229605   0.67766256  0.02695471]
 [ 0.4828611   0.2698627   0.3370565   0.48324526  0.91748013  0.05095007]
 [ 0.80420429  0.16541403  0.78957757  0.02940164  0.4046946   0.96254584]
 [ 0.66934309  0.02629138  0.62714104  0.20828769  0.71963974  0.87280719]
 [ 0.37634699  0.49697003  0.15583049  0.4941665   0.23214339  0.45092006]]
np_a的维度： 2
np_a的形状： (5, 6)
np_a的长度： 30


In [15]:
print("a=\n",np_a)
print("a的维度：",a.dim())
print("a的形状：",a.shape)
print("a的长度：",a.size())
print("a的元素的个数：",a.numel())

a=
 [[ 0.63123599  0.45494402  0.40702602  0.8229605   0.67766256  0.02695471]
 [ 0.4828611   0.2698627   0.3370565   0.48324526  0.91748013  0.05095007]
 [ 0.80420429  0.16541403  0.78957757  0.02940164  0.4046946   0.96254584]
 [ 0.66934309  0.02629138  0.62714104  0.20828769  0.71963974  0.87280719]
 [ 0.37634699  0.49697003  0.15583049  0.4941665   0.23214339  0.45092006]]
a的维度： 2
a的形状： torch.Size([5, 4])
a的长度： torch.Size([5, 4])
a的元素的个数： 20


可以发现：

* NumPy中array的ndim, shape, size都是属性
* PyTorch中Tensor的dim(), size()都是函数，shape为属性
* NumPy中size代表元素的个数，而PyTorch中Tensor元素的个数是使用numel()方法。

## 索引、视图与形状调整与

与NumPy类似，Tensor也可以通过切片等操作产生Tensor的视图（view），与NumPy一样，视图并不会对Tensor进行复制，仅仅是一个引用。与NumPy类似，可以使用切片操作：

In [16]:
a=torch.tensor([1,2,3,4])
print("a=",a)
b=a[0:2]
print("b=",b)
b[0]=3
print("b=",b)
print("a=",a)

a= tensor([1, 2, 3, 4])
b= tensor([1, 2])
b= tensor([3, 2])
a= tensor([3, 2, 3, 4])


跟NumPy类似的是，也可以使用掩码操作，不过掩码操作所创建的是一个拷贝，而非视图：

In [17]:
a=torch.tensor([1,2,3,4])
print("a=",a)
c=a[a>=3]
print("c=",c)
c[-1]=5
print("c=",c)
print("a=",a)
print("b=",b)

a= tensor([1, 2, 3, 4])
c= tensor([3, 4])
c= tensor([3, 5])
a= tensor([1, 2, 3, 4])
b= tensor([3, 2])


如果需要调整Tensor的形状，可以使用Tensor的view()函数以及reshape()方法，其中view()和reshape()接受的参数为修改后的形状，区别在于view()一定会创建视图，而reshape()则不一定，也可能创建原Tensor的拷贝。

In [18]:
a=torch.tensor([1,2,3,4])
print("a=",a)
b=a.view(2,2)
print("b=\n",b)
c=a.reshape(2,2)
print("c=\n",c)
print("现在对b重新赋值")
b[0,0]=1
print("a=",a)
print("b=\n",b)
print("c=\n",c)

a= tensor([1, 2, 3, 4])
b=
 tensor([[1, 2],
        [3, 4]])
c=
 tensor([[1, 2],
        [3, 4]])
现在对b重新赋值
a= tensor([1, 2, 3, 4])
b=
 tensor([[1, 2],
        [3, 4]])
c=
 tensor([[1, 2],
        [3, 4]])


也可以使用torch.reshape()函数：

In [19]:
d=torch.reshape(a,(2,2))
d[0,0]=5
print("a=",a)
print("d=\n",d)

a= tensor([5, 2, 3, 4])
d=
 tensor([[5, 2],
        [3, 4]])


形状也可以使用-1来描述，当使用-1时，自动计算该维度的大小：

In [20]:
a=torch.tensor([1,2,3,4,5,6])
print("a=",a)
b=a.view(-1,2)
print("b=\n",b)

a= tensor([1, 2, 3, 4, 5, 6])
b=
 tensor([[1, 2],
        [3, 4],
        [5, 6]])


可以使用该特性将Tensor转化为向量：

In [21]:
a=torch.tensor([[1,2,3],[4,5,6]])
print("a=\n",a)
b=a.view(-1)
print("b=\n",b)

a=
 tensor([[1, 2, 3],
        [4, 5, 6]])
b=
 tensor([1, 2, 3, 4, 5, 6])


此外还可以使用squeeze()和unsqueeze()方法对某一个维度压缩一个1或者增加一个1，比如如下的Tensor中，第0个维度的个数为1，可以使用squeeze()方法删掉这个维度：

In [22]:
a=torch.tensor([[1,2,3,4,5,6]])
print(a)
print(a.shape)
b=a.squeeze(0)
b

tensor([[1, 2, 3, 4, 5, 6]])
torch.Size([1, 6])


tensor([1, 2, 3, 4, 5, 6])

反过来也可以在指定维度上增加一个维度：

In [23]:
a=torch.tensor([1,2,3,4,5,6])
print(a)
print(a.shape)
b=a.unsqueeze(1)
b

tensor([1, 2, 3, 4, 5, 6])
torch.Size([6])


tensor([[1],
        [2],
        [3],
        [4],
        [5],
        [6]])

## Tensor的计算

与NumPy类似，Tensor也有很多计算功能，这与我们之前所学的NumPy几乎是相同的。

一个与NumPy有点区别的地方是，在Tensor中，很多运算函数都有带下划线的版本，比如Tensor的add()方法同样有add_()方法，两者的区别在于：

* 使用a.add(b)会返回计算结果a+b，而不改变a
* 使用a.add_()会改变a，计算结束后a=a+b

比如：

In [24]:
a=torch.tensor([1,2,3])
b=torch.tensor([4,5,6])
c=a+b
print("c=",c)
print("a=",a)
c=a.add(b)
print("c=",c)
print("a=",a)
a.add_(b)
print("a=",a)

c= tensor([5, 7, 9])
a= tensor([1, 2, 3])
c= tensor([5, 7, 9])
a= tensor([1, 2, 3])
a= tensor([5, 7, 9])


PyTorch中支持NumPy中支持的很多运算，比如：

* 逐元素的算数运算
  * 加（+,add()）
  * 减（-,abs()）
  * 乘（*,mul()）
  * 除（/,div()）
  * 取反（-，neg()）
* 比较运算
  * 大于（>,gt()）、大于等于（>=,ge()）、小于（<,lt()）、小于等于（>,le()）
  * 等于（==, eq()）、不等于（!=,ne()）
  * 排序：sort()
  * 最大值（max()）、最小值（min()）
  * 最大的k个数：topk()
* 函数运算
  * 绝对值（abs()）、平方根（sqrt()）
  * 三角函数：sin(), cos(), asin(), tan(), atan(), cosh(), tanh().....
  * 向上取整（ceil()）、向下取整（floor()）、四舍五入（round()）、取整数部分（trunc()）、取余（fmod()）、符号函数（sign()）
  * 指数（exp()）、对数（log()）
  * 激活函数
    * sigmoid()
    * tanh()
    * softmax()
* 归并函数
  * 均值（mean()）、求和（sum()）、中位数（median()）、众数（mode()）
  * 标准差（std()）、方差（var()）
  * 范数（norm()）、距离（dist()）
  * 累加（cumsum()）、累乘（cumprod()）
* 矩阵运算
  * 内积（dot()）、外积（cross()）
  * 转置（t()）
  * 矩阵乘法（mm()）
  * 矩阵与向量乘法（mv()）
  * 迹（trace()）
  * 求逆（inverse()）
  * 对角线元素（diag()）
  * 奇异值分解（svd()）
  
等等非常多的运算。

比如：

In [25]:
a=torch.tensor([1.,2,10])
b=torch.tensor([2.,9,8])
print("---\na+b=\n",a+b)
print("---\na/b=\n",a/b)
print("---\na>=b\n",a>=b)
print("---\nb.sort()=\n",b.sort())
print("---\ntorch.max(a,b)=\n",torch.max(a,b))
print("---\nb.max()=\n",b.max())
print("---\nb.topk(2)=\n",b.topk(2))
print("---\ntorch.sqrt(b)=\n",torch.sqrt(b))
print("---\ntorch.sigmoid(b)=\n",torch.sigmoid(b))
print("---\na.sum()=\n",a.sum())
print("---\na.std()=\n",a.std())
print("---\na.norm(2)=\n",a.norm(2))
print("---\ntorch.dist(a,b,2)=\n",torch.dist(a,b,2))
print("---\na.cumsum()=\n",a.cumsum(0))

---
a+b=
 tensor([ 3., 11., 18.])
---
a/b=
 tensor([0.5000, 0.2222, 1.2500])
---
a>=b
 tensor([False, False,  True])
---
b.sort()=
 torch.return_types.sort(
values=tensor([2., 8., 9.]),
indices=tensor([0, 2, 1]))
---
torch.max(a,b)=
 tensor([ 2.,  9., 10.])
---
b.max()=
 tensor(9.)
---
b.topk(2)=
 torch.return_types.topk(
values=tensor([9., 8.]),
indices=tensor([1, 2]))
---
torch.sqrt(b)=
 tensor([1.4142, 3.0000, 2.8284])
---
torch.sigmoid(b)=
 tensor([0.8808, 0.9999, 0.9997])
---
a.sum()=
 tensor(13.)
---
a.std()=
 tensor(4.9329)
---
a.norm(2)=
 tensor(10.2470)
---
torch.dist(a,b,2)=
 tensor(7.3485)
---
a.cumsum()=
 tensor([ 1.,  3., 13.])


In [26]:
a=torch.tensor([[1.,2,3],[5,6,4],[8,1,0]])
b=torch.tensor([[2.,4,3],[5,7,1],[8,10,2]])
c=torch.tensor([1,2,3.])
d=torch.tensor([3,2,1.])
print("a=\n",a)
print("b=\n",b)
print("c=\n",c)
print("---\na+b=\n",a+b)
print("---\ntorch.exp(a)=\n",torch.exp(a))
print("---\na.diag()=\n",a.diag())
print("---\na.t()=\n",a.t())
print("---\ntorch.dot(c,d)=\n",torch.dot(c,d))
print("---\ntorch.mm(a,b)=\n",torch.mm(a,b))
print("---\na.mm(b)=\n",a.mm(b))
print("---\na.trace()=",a.trace())
print("---\na.inverse()=",a.inverse())
print("---\na.mm(a.inverse())=",a.mm(a.inverse()))
print("---\ntorch.mv(a,c)=\n",torch.mv(a,c))
print("---\ntorch.mv(a.t(),c)=\n",torch.mv(a.t(),c))

a=
 tensor([[1., 2., 3.],
        [5., 6., 4.],
        [8., 1., 0.]])
b=
 tensor([[ 2.,  4.,  3.],
        [ 5.,  7.,  1.],
        [ 8., 10.,  2.]])
c=
 tensor([1., 2., 3.])
---
a+b=
 tensor([[ 3.,  6.,  6.],
        [10., 13.,  5.],
        [16., 11.,  2.]])
---
torch.exp(a)=
 tensor([[2.7183e+00, 7.3891e+00, 2.0086e+01],
        [1.4841e+02, 4.0343e+02, 5.4598e+01],
        [2.9810e+03, 2.7183e+00, 1.0000e+00]])
---
a.diag()=
 tensor([1., 6., 0.])
---
a.t()=
 tensor([[1., 5., 8.],
        [2., 6., 1.],
        [3., 4., 0.]])
---
torch.dot(c,d)=
 tensor(10.)
---
torch.mm(a,b)=
 tensor([[ 36.,  48.,  11.],
        [ 72., 102.,  29.],
        [ 21.,  39.,  25.]])
---
a.mm(b)=
 tensor([[ 36.,  48.,  11.],
        [ 72., 102.,  29.],
        [ 21.,  39.,  25.]])
---
a.trace()= tensor(7.)
---
a.inverse()= tensor([[ 0.0580, -0.0435,  0.1449],
        [-0.4638,  0.3478, -0.1594],
        [ 0.6232, -0.2174,  0.0580]])
---
a.mm(a.inverse())= tensor([[1.0000e+00, 2.9802e-08, 0.0000e+00],
    

最后，还记得如果我们使用torch.from_numpy()函数和Tensor的numpy()函数在NumPy的array和PyTorch的Tensor之间转换，内存是共享的，转换速度很快，可以直接进行操作而付出较小的性能代价：

In [27]:
a=torch.tensor([[1.,2,3],[5,6,4],[8,1,0]])
b=torch.tensor([[2.,4,3],[5,7,1],[8,10,2]])
print("a=\n",a)
print("b=\n",b)
a_np=a.numpy()
b_np=b.numpy()
c_np=a_np@b_np
c=torch.tensor(c_np)
print("c=\n",c)
print("c_np=\n",c_np)

a=
 tensor([[1., 2., 3.],
        [5., 6., 4.],
        [8., 1., 0.]])
b=
 tensor([[ 2.,  4.,  3.],
        [ 5.,  7.,  1.],
        [ 8., 10.,  2.]])
c=
 tensor([[ 36.,  48.,  11.],
        [ 72., 102.,  29.],
        [ 21.,  39.,  25.]])
c_np=
 [[  36.   48.   11.]
 [  72.  102.   29.]
 [  21.   39.   25.]]


最后，Tensor也支持广播操作，在这里不再赘述。

# 自动求导

在神经网络中，优化算法需要计算损失函数对于每个参数的导数，然而在动辄成千上万甚至数十万参数的情况下，靠人工取计算这些导数显然是不现实的。为此，在现代化的深度学习框架中，都包含了自动求导的模块。

在PyTorch中，torch.autograd模块就是用来实现自动求导的。

对于一个Tensor，在创建的时候可以通过加入"requires_grad=True"选项，表示需要对这个Tensor进行求导。比如，我们对一个最简单的线性函数：$$y=x'b$$进行求导，易得：$$\frac{dy}{db}=x$$一般而言，如果$(x,y)$为数据，$b$为参数，我们需要对参数求导（而非数据），从而我们只需要要求Tensor b在创建时加入requires_grad=True选项就可以了，比如：

In [34]:
x=torch.rand(3)
b=torch.tensor([1.,2,3], requires_grad=True)
y=torch.dot(x,b)
print("x=\n",x)
print("y=",y)
y.backward()
print("dy/dx=\n",x.grad)
print("dy/db=\n",b.grad)

x=
 tensor([0.7134, 0.1564, 0.2105])
y= tensor(1.6577, grad_fn=<DotBackward>)
dy/dx=
 None
dy/db=
 tensor([0.7134, 0.1564, 0.2105])


可以看到由于b加入了requires_grad=True选项，我们可以使用y.backward()选项，实现对b的求导；而x由于没有这个选项，从而没有对x进行求导。

我们还可以使用更加复杂的函数，比如：$$y=x'b \\ z=e^{y}$$其对$b$的导数应该按照链式法则进行计算：$$\frac{dz}{db}=\frac{dz}{dy}\cdot\frac{dy}{db}=e^{y}\cdot x=zx$$

In [35]:
x=torch.rand(3)
b=torch.tensor([1.,2,3], requires_grad=True)
y=torch.dot(x,b)
z=torch.exp(y)
print("x=\n",x)
print("y=",y)
print("z=",z)
z.backward()
print("dz/db=\n",b.grad)

x=
 tensor([0.7400, 0.6846, 0.7948])
y= tensor(4.4936, grad_fn=<DotBackward>)
z= tensor(89.4405, grad_fn=<ExpBackward>)
dz/db=
 tensor([66.1897, 61.2272, 71.0877])


以上复合函数求导在PyTorch中是以 **计算图** 的方式进行的，比如上面的例子我们可以使用如下的计算图来表示：
![](pic/compute_graph.gv.png "计算图")
其中菱形代表计算。在计算导数时，使用一种“反向传播”算法，即使用链式法则，按照计算方向的反方向计算导数，如下图所示：
![](pic/compute_graph_back.gv.png "反向传播图")
上图中虚线代表计算导数的方向，每条虚线计算导数，然后在节点上从最终的节点$z$到该节点$b$的导数累乘起来，就得到了$\frac{dz}{db}$。正因如此，计算节点导数的函数才被成为backward()。

PyTorch的计算图有个非常重要的优势是使用了动态的计算图，意味着每次传播时，计算图可以动态重新构建。

在中间节点加入参数也是完全可行的，比如：$$y=x'b \\ z=e^{cy}$$那么导数为：$$\frac{dz}{dc}=ye^{cy}=yz \\ \frac{dz}{db}=\frac{dz}{dy}\cdot\frac{dy}{db}=ce^{cy}\cdot x=czx$$使用计算图表示为：
![](pic/compute_graph_back2.gv.png "反向传播图")

In [36]:
x=torch.rand(3)
b=torch.tensor([1.,2,3], requires_grad=True)
y=torch.dot(x,b)
c=torch.tensor([2.], requires_grad=True)
z=torch.exp(c*y)
print("x=\n",x)
print("c=",c)
print("y=",y)
print("z=",z)
z.backward()
print("dz/db=\n",b.grad)
print("dz/dc=\n",c.grad)

x=
 tensor([0.1427, 0.4501, 0.1079])
c= tensor([2.], requires_grad=True)
y= tensor(1.3666, grad_fn=<DotBackward>)
z= tensor([15.3830], grad_fn=<ExpBackward>)
dz/db=
 tensor([ 4.3908, 13.8477,  3.3199])
dz/dc=
 tensor([21.0229])


在统计和机器学习中，计算的目标函数（损失函数）通常是一个求和的形式：$$Q(x,b)=\sum_{i=1}^N q(x_i,b)$$此时我们对$b$求导：$$\frac{dQ}{db}=\sum_{i=1}^N \frac{dq(x_i,b)}{db}$$在默认的情况下，每次调用backward()后，导数都会自动进行累加：

In [43]:
x1=torch.tensor([1,2,3.])
x2=torch.tensor([4,5,6.])
b=torch.tensor([1.,2,3], requires_grad=True)
y=torch.dot(x1,b)
y.backward()
print("dy/db=\n",b.grad)
y=torch.dot(x2,b)
y.backward()
print("dy/db=\n",b.grad)

dy/db=
 tensor([1., 2., 3.])
dy/db=
 tensor([5., 7., 9.])


如果不需要累加，需要对梯度清零：

In [59]:
x1=torch.tensor([1,2,3.])
x2=torch.tensor([4,5,6.])
b=torch.tensor([1.,2,3], requires_grad=True)
y=torch.dot(x1,b)
y.backward()
print("dy/db=\n",b.grad)
b.grad.zero_()
y=torch.dot(x2,b)
y.backward()
print("dy/db=\n",b.grad)

dy/db=
 tensor([1., 2., 3.])
dy/db=
 tensor([4., 5., 6.])


在PyTorch中，只允许一个标量对张量求导，而不允许张量对张量求导。如果需要张量对张量求导，需要手动计算。
比如，对于函数：$$y_1=b_1 \times x_1 + b_2 \times x_2 +b_3 \times x_1\times x_2 \\ y_2= b_2\times x_1^{b_1}+x_2^{b_3}$$参数为$\left[b_1,b_2,b_3\right]$，而输出为$\left[y_1,y_2\right]$，其导数为：$$\frac{dy}{db}=\left[\begin{array}{ccc}
x_{1} & x_{2} & x_{1}x_{2}\\
b_{2}x_{1}^{b_{1}}\ln\left(x_{1}\right) & x_{1}^{b_{1}} & x_{2}^{b_{3}}\ln\left(x_{2}\right)
\end{array}\right]$$

为了计算该Jacobian矩阵，可以分开计算第一行和第二行。在backward()函数中，可以加入一个参数向量v，比如如果使用：
```python
y.backward(v)
```

那么该函数就将计算：$\frac{d(vy')}{db}$，我们只需要分别令$v_1=\left[1,0\right],v_2=\left[0,1\right]$就可以分别计算出Jaccobian的两行，然后放在一起即可：

In [60]:
x=torch.tensor([1.,2])
b=torch.tensor([1,2,3.], requires_grad=True)
y=torch.zeros(2)
y[0]=b[0]*x[0]+b[1]*x[1]+b[2]*x[0]*x[1]
y[1]=b[1]*(x[0]**b[0])+x[1]**b[2]
print("x=",x)
print("y=",y)
y.backward(torch.tensor([1.,0]), retain_graph=True)
J1=b.grad
b.grad.zero_()
J2=y.backward(torch.tensor([0,1.]))
J2=b.grad
print("J1=",J1)
print("J2=",J2)
J=torch.zeros(2,3)
J[0]=J1
J[1]=J2
print("J=\n",J)

x= tensor([1., 2.])
y= tensor([11., 10.], grad_fn=<CopySlices>)
J1= tensor([0.0000, 1.0000, 5.5452])
J2= tensor([0.0000, 1.0000, 5.5452])
J=
 tensor([[0.0000, 1.0000, 5.5452],
        [0.0000, 1.0000, 5.5452]])


注意以上代码中的两个细节：

* 第一次调用backward时，使用了retain_graph=True选项，该选项代表在多次传播时，强制保留反向传播中的中间计算结果
* 由于每一次传播时grad会累加，为了不让其累加，我们对b.grad进行了清零操作

# 数据读取

在torch.utils.data模块中包含了读取数据的类，比如Dataset、IterableDataset和DataLoader等。其中：

* Dataset提供了一个抽象类，用于用户自定义数据集，用户需要重载两个函数完成这个类：
    * \_\_len\_\_()：数据量大小
    * \_\_getitem\_\_()：获得一条数据
* IterableDataset同样提供了一个抽象类，与Dataset不同的是该抽象类使用迭代器的方式返回数据，通常用来比较大型的数据或者从数据库、远程服务器中读取的数据等等，用户需要重载一个函数完成这个类：
    * \_\_iter\_\_()：返回一个数据条目（可迭代）
* DataLoader在Dataset的基础上定义一个迭代器，实现批量（batch）读取、随机读取等操作。

我们首先以一个伪数据看一下Dataset的使用方法：

In [66]:
import numpy as np
# 这里假设一个伪数据集，使用NumPy生成
X=np.random.normal(0,1,(10,3))
b=np.array([1,2,3.])
Y=X@b.T
print("X=\n",X,"\nY=\n",Y)

X=
 [[ 0.58602222  0.80495029  0.19734014]
 [-1.57468075  0.81411451  1.9698675 ]
 [-0.93712474  1.02467448 -1.11080625]
 [-2.02342489 -0.32069881 -0.12281846]
 [ 0.32559092 -0.74437519 -1.49184672]
 [-1.19720758  0.34140826 -0.44582146]
 [ 0.02193199  0.75287457  0.1785099 ]
 [ 1.2688634   0.26122625  3.94300985]
 [-1.45330455 -0.20342525  0.12963423]
 [ 0.65416681 -0.47964503  0.65412619]] 
Y=
 [  2.78794322   5.96315078  -2.22019451  -3.03327789  -5.6386996
  -1.85185543   2.06321083  13.62034545  -1.47125234   1.65725532]


10

以上使用NumPy产生了10个观测，其中有3个特征，以及一个标签（Y），接下来我们定义Dataset：

In [74]:
from torch.utils.data import Dataset
class fake_data(Dataset):
    def __len__(self):
        return X.shape[0]
    def __getitem__(self, i):
        x=X[i,:]
        y=Y[i]
        data=torch.from_numpy(x)
        label=torch.tensor(y)
        return data, label

以上我们就定义了一个Dataset，我们可以使用如下方法获得给定下标的数据：

In [75]:
fd=fake_data()
print("共有{}个观测".format(fd.__len__()))
print("第1个观测为：\n",fd[0])

共有10个观测
第1个观测为：
 (tensor([0.5860, 0.8050, 0.1973], dtype=torch.float64), tensor(2.7879))


以上的Dataset只提供了一个简单的数据抽取对象，而深度学习所需要的更多的操作可以由DataLoader来完成。

为了创建一个DataLoader，需要提供以下信息：
* Dataset：像上面定义的Dataset，数据源
* shuffle：是否将数据打乱
* batch_size: 批量处理每批的大小
* sampler/batch_sampler：一个用于将数据打乱的类，可以使用默认
* num_workers：载入数据使用的进程数，默认为0，即使用主进程
* pin_memory：是否将数据保存在CUDA的pinned memory区，从而放入GPU中会快一些
* drop_last：是否扔掉最后一个不完整的batch（通常由于数据量/批大小不能整除）

比如：

In [78]:
from torch.utils.data import DataLoader
fdl=DataLoader(fd, shuffle=True, batch_size=3, drop_last=True)
for d,l in enumerate(fdl):
    print("第{}批：".format(d))
    print(l[0])
    print(l[1])

第0批：
tensor([[ 0.0219,  0.7529,  0.1785],
        [ 0.5860,  0.8050,  0.1973],
        [-1.5747,  0.8141,  1.9699]], dtype=torch.float64)
tensor([2.0632, 2.7879, 5.9632])
第1批：
tensor([[-2.0234, -0.3207, -0.1228],
        [ 0.6542, -0.4796,  0.6541],
        [ 0.3256, -0.7444, -1.4918]], dtype=torch.float64)
tensor([-3.0333,  1.6573, -5.6387])
第2批：
tensor([[-0.9371,  1.0247, -1.1108],
        [-1.1972,  0.3414, -0.4458],
        [ 1.2689,  0.2612,  3.9430]], dtype=torch.float64)
tensor([-2.2202, -1.8519, 13.6203])


如果需要全部返回，设定batch_size为数据量即可：

In [83]:
fdl=DataLoader(fd, shuffle=True, batch_size=fd.__len__(), drop_last=True)
for d in fdl:
    print(d[0])
    print(d[1])

tensor([[ 0.6542, -0.4796,  0.6541],
        [ 0.0219,  0.7529,  0.1785],
        [-2.0234, -0.3207, -0.1228],
        [ 0.5860,  0.8050,  0.1973],
        [-1.4533, -0.2034,  0.1296],
        [-0.9371,  1.0247, -1.1108],
        [ 1.2689,  0.2612,  3.9430],
        [ 0.3256, -0.7444, -1.4918],
        [-1.5747,  0.8141,  1.9699],
        [-1.1972,  0.3414, -0.4458]], dtype=torch.float64)
tensor([ 1.6573,  2.0632, -3.0333,  2.7879, -1.4713, -2.2202, 13.6203, -5.6387,
         5.9632, -1.8519])


# 应用实例：使用PyTorch计算Logistic回归

应用PyTorch可以自动计算导数的特性，我们可以轻松的计算线性回归、Logistic回归等多数简单的统计模型。接下来我们就使用PyTorch计算Logistic回归，并与真实概率进行比较。

首先，我们在一个fake数据集上进行计算，并验证。